In [37]:
import numpy
import pandas as pd
import tensorflow as tf
import os

from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import ModelCheckpoint
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

# seed 값 설정
seed = 0
numpy.random.seed(seed)
tf.random.set_seed(seed)

df = pd.read_csv("/content/drive/MyDrive/080228-master/deeplearning/dataset/sonar.csv", header=None)
dataset = df.values
X = dataset[:, 0:60].astype(float) # X = dataset[:, 0:60] -> float으로 타입 변경 필요
Y_obj = dataset[:, 60] # 60번째 열은 string

# 문자를 0이나 1로 변환
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)

# 5개의 파일로 쪼갬
n_fold = 5

skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

# 빈 accuracy 배열
accuracy = []

# 모델의 설정, 컴파일, 실행
index = 1
for train, test in skf.split(X, Y): # 반복문은 n_fold의 개수만큼 반복
  model = Sequential()
  model.add(Dense(24, input_dim=60, activation='relu'))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

  # 모델 저장 폴더 설정
  MODEL_DIR = "./model1/"
  if not os.path.exists(MODEL_DIR): # 위의 폴더가 존재하지 않으면
    os.mkdir(MODEL_DIR) # 이 이름의 폴더를 만들어 줌

  # 모델 실행 및 저장 (학습은 학습셋으로 -> train 사용)
  history = model.fit(X[train], Y[train], epochs=100, batch_size=5)
  k_accuracy = "%.4f" % (model.evaluate(X[test], Y[test])[1])
  accuracy.append(k_accuracy)

  # 모델 저장 조건 설정
  modelpath="./model1/lab7-model-" + str(index) + ".hdf5"
  model.save(modelpath)

  # index 증가
  index += 1

# 결과 출력
print("\n % .f fold accuracy:" % n_fold, accuracy)

Mounted at /content/drive
Epoch 1/100
34/34 [==============================] - 1s 4ms/step - loss: 0.2374 - accuracy: 0.5783
Epoch 2/100
34/34 [==============================] - 0s 4ms/step - loss: 0.2286 - accuracy: 0.6867
Epoch 3/100
34/34 [==============================] - 0s 4ms/step - loss: 0.2201 - accuracy: 0.6687
Epoch 4/100
34/34 [==============================] - 0s 3ms/step - loss: 0.2128 - accuracy: 0.7530
Epoch 5/100
34/34 [==============================] - 0s 4ms/step - loss: 0.2031 - accuracy: 0.8133
Epoch 6/100
34/34 [==============================] - 0s 3ms/step - loss: 0.1920 - accuracy: 0.8133
Epoch 7/100
34/34 [==============================] - 0s 3ms/step - loss: 0.1828 - accuracy: 0.7771
Epoch 8/100
34/34 [==============================] - 0s 4ms/step - loss: 0.1732 - accuracy: 0.8373
Epoch 9/100
34/34 [==============================] - 0s 6ms/step - loss: 0.1618 - accuracy: 0.8133
Epoch 10/100
34/34 [==============================] - 0s 6ms/step - loss: 0.1543 - 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


34/34 [==============================] - 1s 3ms/step - loss: 0.2475 - accuracy: 0.5241
Epoch 2/100
34/34 [==============================] - 0s 3ms/step - loss: 0.2416 - accuracy: 0.6084
Epoch 3/100
34/34 [==============================] - 0s 3ms/step - loss: 0.2351 - accuracy: 0.6446
Epoch 4/100
34/34 [==============================] - 0s 3ms/step - loss: 0.2312 - accuracy: 0.7169
Epoch 5/100
34/34 [==============================] - 0s 3ms/step - loss: 0.2240 - accuracy: 0.7169
Epoch 6/100
34/34 [==============================] - 0s 3ms/step - loss: 0.2162 - accuracy: 0.7590
Epoch 7/100
34/34 [==============================] - 0s 3ms/step - loss: 0.2098 - accuracy: 0.6867
Epoch 8/100
34/34 [==============================] - 0s 3ms/step - loss: 0.2026 - accuracy: 0.7470
Epoch 9/100
34/34 [==============================] - 0s 3ms/step - loss: 0.1905 - accuracy: 0.7831
Epoch 10/100
34/34 [==============================] - 0s 3ms/step - loss: 0.1833 - accuracy: 0.7530
Epoch 11/100
34/34 [=

In [38]:
max_index = accuracy.index(max(accuracy)) # 가장 높은 정확도를 가진 index 추출
print("Max index:", max_index)

modelpath="./model1/lab7-model-" + str(max_index) + ".hdf5"
model = load_model(modelpath)

# 앞의 5행 추출
test_X = X[:5, :]
test_Y = Y[:5]

result = model.predict(test_X)
print("\nLoad %d-th model having %s accuracy" % (max_index, accuracy[max_index]))
print("Sample test of the first 5 element")

# 앞의 5행에 대한 예측값과 실제값 출력
for i in range(5):
  print("index: %d, actual: %.2f, expect: %.2f" % (i, test_Y[i], result[i][0]))

Max index: 3
1/1 [==============================] - 0s 64ms/step

Load 3-th model having 0.8780 accuracy
Sample test of the first 5 element
index: 0, actual: 1.00, expect: 0.66
index: 1, actual: 1.00, expect: 0.93
index: 2, actual: 1.00, expect: 0.86
index: 3, actual: 1.00, expect: 0.80
index: 4, actual: 1.00, expect: 0.90
